# Transformers

In [5]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis')
classifier('I love sunny days!')

/Users/qingyunwang/Dropbox/job/TheDataIncubator/capstone/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

# Nltk engine

In [13]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/qingyunwang/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [118]:
def load_csv(path):
    data = pd.read_csv(path, index_col=0)
    return data

def get_raw_scored(path):
    sia = SentimentIntensityAnalyzer()
    data = load_csv(path)
    data['score'] = data['Text'].apply(sia.polarity_scores).apply(lambda x: x['compound'])
    return data

In [119]:
df = get_raw_scored('reddit_company.csv')

In [120]:
df

,Company,Date,Text,score
0,Apple,2023-11-07,Book recommendation Hey start learn bit goal t...,0.9961
1,Apple,2023-11-08,Understanding Options Michael Sincere Selling ...,0.4019
2,Apple,2023-11-09,US v apple saving high yield Hello heard curre...,0.9969
3,Apple,2023-11-10,Borderline incoherent Well do Apple moat innov...,0.9201
4,Apple,2023-11-11,Wow AAPL blew past show much buying,0.5859
5,Apple,2023-11-12,guy think Apple position feel apple real nice ...,0.9993
6,Apple,2023-11-13,Strong buy signal money apple make sell phone ...,0.9819
7,Apple,2023-11-14,Apple get Google search revenue Safari Alphabe...,0.9942
8,Apple,2023-11-15,crazy idea think Google pay Apple want win dea...,0.9991
9,Apple,2023-11-16,amount research scream fomo Honestly consider ...,0.9967


In [121]:
df.groupby(['Company'])['score'].mean()

Company
Apple       0.691472
Bilibili   -0.152133
Merck       0.260060
Name: score, dtype: float64

In [124]:
def get_ewm_score(df, alpha_value = 0.3):
    # Function to apply EMA on each group and return a single value
    def calculate_ema(group, alpha=alpha_value):
        return group.ewm(alpha=alpha).mean().iloc[-1]

    # Group by 'col1' and apply the EMA function to 'col2'
    ema_result = df.groupby('Company')['score'].apply(calculate_ema)
    return ema_result

In [125]:
get_ewm_score(df)

Company
Apple       0.268120
Bilibili   -0.018002
Merck       0.081289
Name: score, dtype: float64